### Packages

In [1]:
from typing import Any, List
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data

In [2]:
df = pd.read_csv("../data/dataset.csv", sep=";")
df.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,1,1,1,1,0,0,0,178839,9.653333,1.0
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1.0
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1.0
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99976 entries, 0 to 99975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 99976 non-null  object 
 1   default                              89976 non-null  float64
 2   account_amount_added_12_24m          99976 non-null  int64  
 3   account_days_in_dc_12_24m            88140 non-null  float64
 4   account_days_in_rem_12_24m           88140 non-null  float64
 5   account_days_in_term_12_24m          88140 non-null  float64
 6   account_incoming_debt_vs_paid_0_24m  40661 non-null  float64
 7   account_status                       45603 non-null  float64
 8   account_worst_status_0_3m            45603 non-null  float64
 9   account_worst_status_12_24m          33215 non-null  float64
 10  account_worst_status_3_6m            42274 non-null  float64
 11  account_worst_status_6_12m  

### Identifying Variables and Targets

- Unique identifier: `uuid`
- Target: `default`
- Potential Features: all other columns.

In [4]:
target = "default"
unique_id = "uuid"

Since there are some missing entries for the target variable, we need to drop these entries from the dataset.

In [5]:
df = df.dropna(subset=[target])
df.shape

(89976, 43)

### EDA

+ Gathering features according to their intrinsic type

In [6]:
all_fields = list(filter(lambda x: x not in [target, unique_id], df.columns))

cat_features = list(filter(lambda x: "status" in x, all_fields))
bool_features = ["has_paid"]
string_features = list(filter(lambda x: df[x].dtype.name == "object", all_fields))
numeric_features = list(
    set(all_fields) -
    set(cat_features) - 
    set(bool_features) - 
    set(string_features)
)

feature_groups = {
    "category": cat_features,
    "bool": bool_features,
    "str": string_features,
    "float64": numeric_features,
}

+ Converting them to their final type

In [7]:
def convert_type(df: pd.DataFrame, cols: List[str], target_type: Any) -> pd.DataFrame:
    return df[cols].astype(target_type)

for feat_type, feat_group in feature_groups.items():
    df[feat_group] = convert_type(df, feat_group, feat_type)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89976 entries, 0 to 89975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   uuid                                 89976 non-null  object  
 1   default                              89976 non-null  float64 
 2   account_amount_added_12_24m          89976 non-null  float64 
 3   account_days_in_dc_12_24m            79293 non-null  float64 
 4   account_days_in_rem_12_24m           79293 non-null  float64 
 5   account_days_in_term_12_24m          79293 non-null  float64 
 6   account_incoming_debt_vs_paid_0_24m  36619 non-null  float64 
 7   account_status                       41042 non-null  category
 8   account_worst_status_0_3m            41042 non-null  category
 9   account_worst_status_12_24m          29921 non-null  category
 10  account_worst_status_3_6m            38038 non-null  category
 11  account_worst_s

##### Target

This is a highly imbalanced classification problem - as we would expect from a default prediction.

In [9]:
df[target].value_counts()

0.0    88688
1.0     1288
Name: default, dtype: int64

In [10]:
df[target].value_counts() / len(df)

0.0    0.985685
1.0    0.014315
Name: default, dtype: float64

##### Category Features

##### Boolean Features

##### String Features

##### Numeric Features